In [15]:
import openfoamparser as ofp

In [16]:
case = ofp.FoamMesh("of_polymesh/")

In [17]:
case.boundary

{b'inlet': Boundary(type=b'patch', num=75, start=11100, id=-10),
 b'sym': Boundary(type=b'patch', num=25, start=11175, id=-11),
 b'wedge': Boundary(type=b'patch', num=50, start=11200, id=-12),
 b'top': Boundary(type=b'patch', num=75, start=11250, id=-13),
 b'back': Boundary(type=b'empty', num=5625, start=11325, id=-14),
 b'front': Boundary(type=b'empty', num=5625, start=16950, id=-15),
 b'outlet': Boundary(type=b'patch', num=75, start=22575, id=-16)}

In [18]:
# %%
import os as os

import case_creator as cc
import numpy as np

case_path = ""

cc.initCase(case_path)
mesh = cc.readOpenFoamMesh("of_polymesh")
cc.writeOpenFoamMesh(case_path, mesh)

cc.writeFluidProperties(
    case_path,
    R=287.1,
    gamma=1.4,
    Pr=0.713,
    c_p=1005,
    c_v=717.5,
    reference_p=1.0,
    reference_v=1.0,
    reference_T=1.0,
    reference_mu=0.0,
    reference_k=1.0,
)

cc.writeSystemControls(
    case_path,
    n_threads=16,
    save_every_n_steps=10,
    n_sim_steps=5000,
    time_integration_scheme=cc.TimeIntegrationScheme.exp_multi_rk_1_3,
    use_local_time_stepping=True,
    auto_calc_time_step=True,
    dt=1.0,
    cfl=1.0,
    local_dt_C_factor=2.0,
    limiter_type=cc.LimiterType.venkatakrishnan,
    limiter_k_factor=1.0,
    convective_flux_scheme=cc.ConvectiveFluxScheme.riemann_hllc,
    viscous_flux_scheme=cc.ViscousFluxScheme.none,
)

p_init = 101325
v_init = np.array([1041.263, 0.0, 0.0], dtype=np.float64)
T_init = 300

cc.writeInitialConditions(
    case_path,
    mesh,
    p_init,
    v_init,
    T_init,
)

In [19]:
bc_types = [
    cc.BCType.inlet_supersonic,
    cc.BCType.symmetry,
    cc.BCType.wall_inviscid_heated,
    cc.BCType.outlet_supersonic,
    cc.BCType.symmetry,
    cc.BCType.symmetry,
    cc.BCType.outlet_supersonic,
]

bc_p_values = np.array([101325, 0, 0, 0, 0, 0, 0], dtype=np.float64)
bc_grad_p_normal_values = np.array([0, 0, 0, 0, 0, 0, 0], dtype=np.float64)

bc_T_values = np.array([300, 0, 0, 0, 0, 0, 0], dtype=np.float64)
bc_grad_T_normal_values = np.array([0, 0, 0, 0, 0, 0, 0], dtype=np.float64)

bc_v_values = np.array(
    [
        [1041.263, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
    dtype=np.float64,
)

bc_grad_v_normal_values = np.array(
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
    dtype=np.float64,
)

cc.writeBoundaryConditions(
    case_path,
    bc_types,
    bc_p_values,
    bc_v_values,
    bc_T_values,
    bc_grad_p_normal_values,
    bc_grad_v_normal_values,
    bc_grad_T_normal_values,
)